### get geography information (area and land area) from Census Tigerweb, pull in population and housing Decennial Census counts and calculate population and housing density in 2010 and 2020

In [1]:
import pandas as pd
import numpy as np

In [2]:
import geo_getter as get

## urban villages - blocks

In [3]:
blk_10 = pd.read_csv('../data/geo/blk_vil_10_reduced.csv')
blk_20 = pd.read_csv('../data/geo/blk_vil_20_reduced.csv')

blk_10 = blk_10[['name','land_acre','aland']].copy()
blk_10 = blk_10.groupby(['name']).sum().reset_index()
blk_10.rename(columns={'land_acre':'AREALAND_ACRE10','aland':'AREALAND10'},\
                        inplace=True)

blk_20 = blk_20[['name','land_acre','aland']].copy()
blk_20 = blk_20.groupby(['name']).sum().reset_index()
blk_20.rename(columns={'land_acre':'AREALAND_ACRE20','aland':'AREALAND20'},\
                        inplace=True)

In [4]:
blk = pd.merge(blk_10,blk_20,how='left',on='name')

bg_10 = pd.read_csv('../data/geo/bgp_vil_10.csv')
bg_20 = pd.read_csv('../data/geo/bgp_vil_20.csv')

uv10 = bg_10[['name','land_acre','aland']].copy()
uv10 = uv10.groupby(['name']).sum().reset_index()
uv10.rename(columns={'land_acre':'AREALAND_ACRE10','aland':'AREALAND10'},\
                        inplace=True)

uv20 = bg_20[['name','land_acre','aland']].copy()
uv20 = uv20.groupby(['name']).sum().reset_index()
uv20.rename(columns={'land_acre':'AREALAND_ACRE20','aland':'AREALAND20'},\
                        inplace=True)

uv = pd.merge(uv10,uv20,how='left',on='name')

## cities

In [5]:
cities = {455000:'Phoenix',473000:'Tempe',465000:'Scottsdale',427820:'Glendale',\
          4865000:'San Antonio', 4819000:'Dallas',1235000:'Jacksonville',\
          1836000:'Indianapolis'}

geoids = list(cities.keys())

In [6]:
pl10 = get.make_city('2010')
pl20 = get.make_city('2020')

In [7]:
places = pd.merge(pl10,pl20,how='left',on='GEOID')

In [8]:
places = places[places.GEOID.isin(list(cities.keys()))].copy()

In [9]:
places.rename(columns={'BASENAME10':'name'},inplace=True)
places = places.drop(['GEOID','AREAWATER10','BASENAME20','AREAWATER20'],axis=1)

## maricopa

In [10]:
mar10 = get.make_maricopa('2010')
mar20 = get.make_maricopa('2020')

In [11]:
mar = pd.merge(mar10,mar20,how='left',on='GEOID')

In [12]:
mar.rename(columns={'BASENAME10':'name'},inplace=True)
mar = mar.drop(['GEOID','AREAWATER10','BASENAME20','AREAWATER20'],axis=1)

## arizona

In [13]:
az10 = get.make_az('2010')
az20 = get.make_az('2020')
az = pd.merge(az10,az20,how='left',on='GEOID')
az['name'] = 'AZ'

In [14]:
az = az.drop(['GEOID','AREAWATER10','BASENAME10','BASENAME20','AREAWATER20'],axis=1)

## us

In [15]:
us10 = get.make_us('2010')
us20 = get.make_us('2020')
us = pd.merge(us10,us20,how='left',on='FUNCSTAT')

In [16]:
us['name'] = 'US'
us = us.drop(['FUNCSTAT','AREAWATER10','AREAWATER20'],axis=1)

## make big geography table - no urban vil

In [17]:
geos = pd.concat([blk,places,mar,az,us])

## read in pop and housing from output folder

In [18]:
data = pd.read_csv('output/dec_pop_hou_race.csv')

In [19]:
data = data[['name','Hou_10E','Hou_20E','Pop_10E','Pop_20E']]

## calc housing and pop density in 2010 and 2020

In [20]:
geos = pd.merge(data,geos,how='left',on='name')

In [21]:
geos['HouDen_10'] = geos.Hou_10E / geos.AREALAND_ACRE10
geos['PopDen_10'] = geos.Pop_10E / geos.AREALAND_ACRE10
geos['HouDen_20'] = geos.Hou_20E / geos.AREALAND_ACRE20
geos['PopDen_20'] = geos.Pop_20E / geos.AREALAND_ACRE20

In [22]:
geos

,name,Hou_10E,Hou_20E,Pop_10E,Pop_20E,AREALAND_ACRE10,AREALAND10,AREALAND_ACRE20,AREALAND20,HouDen_10,PopDen_10,HouDen_20,PopDen_20
0,Ahwatukee Foothills,33508,34523,77310,80596,2.419149e+04,9.789964e+07,2.406718e+04,9.739658e+07,1.385115,3.195752,1.434443,3.348793
1,Alhambra,53144,53575,128662,146541,1.230369e+04,4.979134e+07,1.230680e+04,4.980391e+07,4.319354,10.457188,4.353286,11.907324
2,Camelback East,70649,75480,134960,145061,2.379485e+04,9.629449e+07,2.380827e+04,9.634882e+07,2.969088,5.671816,3.170326,6.092882
3,Central City,22281,26082,56547,60516,1.347869e+04,5.454640e+07,1.348089e+04,5.455532e+07,1.653054,4.195290,1.934738,4.489021
4,Deer Valley,70062,74894,164332,180505,3.502340e+04,1.417349e+08,3.531692e+04,1.429227e+08,2.000434,4.692063,2.120627,5.111007
5,Desert View,19414,26444,42454,57998,4.283464e+04,1.733459e+08,4.195634e+04,1.697915e+08,0.453231,0.991114,0.630274,1.382342
6,Encanto,27582,30134,54523,58304,6.594518e+03,2.668711e+07,6.594972e+03,2.668895e+07,4.182565,8.267928,4.569239,8.840674
7,Estrella,22674,24836,81483,93616,2.374420e+04,9.608951e+07,2.379467e+04,9.629378e+07,0.954928,3.431701,1.043763,3.934326
8,Laveen,14210,17059,43171,61095,1.434037e+04,5.803351e+07,1.448084e+04,5.860198e+07,0.990909,3.010452,1.178039,4.219023
9,Maryvale,63681,65239,204560,222119,1.947151e+04,7.879851e+07,1.931264e+04,7.815559e+07,3.270471,10.505608,3.378047,11.501225


In [23]:
geos.to_csv('output/dec_pop_hou_den.csv',index=False)